In [ ]:
final_output_path = "D:\\Projects\\SDC\\Term3\\Capstone-Project-SDC-Term3-P3-Udacity\\site-recs\\final_train"
generator_train_path = "D:\\Projects\\SDC\\Term3\\Capstone-Project-SDC-Term3-P3-Udacity\\site-recs\\generator_train"
generator_validation_path = "D:\\Projects\\SDC\\Term3\\Capstone-Project-SDC-Term3-P3-Udacity\\site-recs\\generator_validation"
num_classes = 3

In [ ]:
# filter warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# keras imports
from keras.applications.vgg16 import VGG16, preprocess_input as vgg16_preprocess
from keras.applications.vgg19 import VGG19, preprocess_input as vgg19_preprocess
from keras.applications.xception import Xception, preprocess_input as xception_preprocess
from keras.applications.resnet50 import ResNet50, preprocess_input as res_preprocess
from keras.applications.mobilenet import MobileNet, preprocess_input as mobile_preprocess
# from keras.applications.mobilenetv2 import MobileNetV2, mobile_v2_preprocess
from keras.applications.inception_v3 import InceptionV3, preprocess_input as inception_preprocess
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Input, Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization, Conv2D, MaxPooling2D
from keras import optimizers
from keras import regularizers

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import cv2
import h5py
import os
import json
import datetime
import time

In [ ]:
# load the user configs
with open('config.json') as f:    
  config = json.load(f)

# config variables
model_name    = config["model"]
weights     = config["weights"]
include_top   = config["include_top"]
train_path    = config["train_path"]
features_path   = config["features_path"]
labels_path   = config["labels_path"]
test_size     = config["test_size"]
results     = config["results"]
model_path    = config["model_path"]
# num_classes   = config["num_classes"]
classifier_path = config["classifier_path"]


In [ ]:
from keras import backend as K
K.get_session().close()

In [ ]:
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

In [ ]:
# start time
print ("[STATUS] start time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))
start = time.time()
print(f"Model name: {model_name}")

# create the pretrained models
# check for pretrained weight usage or not
# check for top layers to be included or not
if model_name == "vgg16":
    base_model = VGG16(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('fc1').output)
    preprocess = vgg16_preprocess
    image_size = (224, 224)
elif model_name == "vgg19":
    base_model = VGG19(weights=weights, include_top=False)
#     model = Model(input=base_model.input, output=base_model.get_layer('fc1').output)
    # Creating dictionary that maps layer names to the layers
#     layer_dict = dict([(layer.name, layer) for layer in base_model.layers])

#     # Getting output tensor of the last VGG layer that we want to include
#     x = layer_dict['block2_pool'].output

#     # Stacking a new simple convolutional network on top of it    
#     x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
#     x = MaxPooling2D(pool_size=(2, 2))(x)
#     x = Flatten()(x)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    preprocess = vgg19_preprocess
    image_size = (224, 224)
elif model_name == "resnet50":
    base_model = ResNet50(weights=weights, include_top=include_top)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    preprocess = res_preprocess
    image_size = (224, 224)
elif model_name == "inceptionv3":
    base_model = InceptionV3(include_top=include_top, weights=weights, input_tensor=Input(shape=(299,299,3)))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    preprocess = inception_preprocess
    image_size = (299, 299)
elif model_name == "inceptionresnetv2":
    base_model = InceptionResNetV2(include_top=include_top, weights=weights, input_tensor=Input(shape=(299,299,3)))
    model = Model(input=base_model.input, output=base_model.get_layer('custom').output)
    image_size = (299, 299)
elif model_name == "mobilenet":
    base_model = MobileNet(include_top=include_top, weights=weights, input_tensor=Input(shape=(224,224,3)), input_shape=(224,224,3))
    image_size = (224, 224)
    preprocess = mobile_preprocess
    x = base_model.output
    x = Flatten()(x)
elif model_name == "mobilenetv2":
    base_model = MobileNetV2(include_top=include_top, weights=weights, input_tensor=Input(shape=(224,224,3)), input_shape=(224,224,3))  
    image_size = (224, 224)
    preprocess = mobile_v2_preprocess
elif model_name == "xception":
    base_model = Xception(weights=weights, include_top=include_top)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    image_size = (299, 299)
    preprocess = xception_preprocess
else:
    base_model = None

# Freeze the layers which you don't want to train.
# for layer in base_model.layers[:5]:
for layer in base_model.layers:    
    layer.trainable = False

x = Dense(512, activation="relu", kernel_regularizer=regularizers.l2(0.01))(x)
x = Dropout(0.2)(x)
x = Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.01))(x)
x = Dropout(0.5)(x)

# x = BatchNormalization()(x)
# x = Dropout(0.2)(x)

predictions = Dense(num_classes, activation="softmax")(x)

# creating the final model 
model_final = Model(inputs = base_model.input, outputs = predictions)
# for layer in model_final.layers:    
#     print(layer.trainable)
model_final.summary()

# compile the model 
adam_optimizer = optimizers.Adam(lr=0.001)
model_final.compile(
    loss = "categorical_crossentropy", 
    #optimizer = optimizers.SGD(lr=0.0001, momentum=0.9, nesterov=True), 
    optimizer = adam_optimizer,
    metrics=["accuracy"])

print ("[INFO] successfully loaded base model and model...")

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

def rescale_preprocess(x, data_format=None, mode='caffe'):
#     x = x[0:140 , 0:224, :]
    x = cv2.resize(x, dsize=image_size, interpolation=cv2.INTER_CUBIC)
    return preprocess(x, data_format, mode)

datagen = ImageDataGenerator(
    preprocessing_function=rescale_preprocess,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0,
    width_shift_range=0.02,
    height_shift_range=0.05,
    rotation_range=3,
    validation_split=0.2)

img_width, img_height = image_size[0], image_size[1]
batch_size = 4
train_generator = datagen.flow_from_directory(
    final_output_path, 
    target_size = (img_height, img_width),
    batch_size = batch_size,
#     save_to_dir=generator_train_path,
    shuffle=True,
    subset='training')
print(train_generator.class_indices)
validation_generator  = datagen.flow_from_directory(
    final_output_path, 
    target_size = (img_height, img_width),
    batch_size = batch_size,
#     save_to_dir=generator_validation_path,
    shuffle=True,
    subset='validation')
print(validation_generator.class_indices)

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

# Save the model according to the conditions  
checkpoint = ModelCheckpoint(model_name + "_site.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')

In [ ]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [ ]:
if not os.path.exists(generator_train_path):
    os.makedirs(generator_train_path)
else:
    output_file_list = glob.glob(os.path.join(generator_train_path, "*.*"))
    for f in output_file_list:
        os.remove(f)

if not os.path.exists(generator_validation_path):
    os.makedirs(generator_validation_path)
else:
    output_file_list = glob.glob(os.path.join(generator_validation_path, "*.*"))
    for f in output_file_list:
        os.remove(f)

In [ ]:
# Train the model
epochs = 100
hist = model_final.fit_generator(
    generator=train_generator,
    epochs = epochs,
    verbose = 1,
    validation_data = validation_generator,
    callbacks = [checkpoint, early])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from random import shuffle
import datetime
import glob
import os
import cv2

show_figures = False

input_folder="D:/Projects/SDC/Term3/Capstone-Project-SDC-Term3-P3-Udacity/site-recs/final_train/green"
image_paths = glob.glob(os.path.join(input_folder, "*.jpg"))
shuffle(image_paths)

model_final.load_weights(model_name + "_site.h5")

start_time = datetime.datetime.now()
nr_images = 1000

rows = 20

if show_figures:
    fig, axes = plt.subplots(nrows=rows, ncols=1, figsize=(28, rows * 8))

print ("[INFO] program started on - " + str(start_time))
# labels = [list(train_generator.class_indices.keys())]
labels = ['green', 'no', 'red']
print(labels)

bad_samples = 0
for i, image_name in enumerate(image_paths[:nr_images]):
    im = cv2.imread(image_name)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = im.astype(np.float32)
    im = cv2.resize(im, image_size, interpolation = cv2.INTER_CUBIC)  
    im = preprocess(im)
    im = np.expand_dims(im, axis =0)
    prob = model_final.predict(im)
    probs = prob[0]
#     print(probs)
    j = np.argmax(prob,axis=1)[0]
    
    if j != 0:
        if show_figures and bad_samples < rows:
            img=mpimg.imread(image_name)
            axes[bad_samples].imshow(img)
            axes[bad_samples].set_title(f'Best guess: {labels[j]} with certainty {probs[j]}')
        bad_samples += 1

end_time = datetime.datetime.now()
time_diff = end_time - start_time
print (f"Time to run: {time_diff}")

print ("Bad samples %: " + str((float)(bad_samples) * 100 / nr_images))